In [ ]:
# 这是 Tiny Imagenet 数据集的预处理
from torch.utils.data import Dataset
import os
import pandas as pd

class TinyImagenet(Dataset):
    def __init__(self, root, type):
        super().__init__()
        self.root = root
        self.type = type
        
        if not os.path.exists(root):
            raise ValueError("dataset path not right!")
        if type not in ['train', 'val']:
            raise ValueError("dataset type not right!")
        
        self.annotations_file = self.get_annotations_file(type)

    def get_annotations_file(self, type):
        df = pd.DataFrame(columns=['img_name', 'label'])
        rows = []
        mapping = {}
        with open(os.path.join(self.root, 'wnids.txt'), 'r') as f:
            lines = f.readlines()
            lines = [line.strip() for line in lines]
            for idx, line in enumerate(lines):
                mapping[line] = idx

        if type == 'train':
            '''
            去遍历 train 下面的每个子文件夹
                对于每个子文件夹, 通过文件夹名 mapping 找到对应的 label
                对于每个子文件夹, 进入其下的 txt, 把第一列读出来, 作为 img_names, 然后把所有 pairs 存到 rows 里
            '''
            path_train = os.path.join(self.root, 'train')
            for entry in os.listdir(path_train):
                label = mapping[entry]
                with open(os.path.join(path_train, entry, entry + '_boxes.txt'), 'r') as f:
                    lines = f.readlines()
                    for line in lines:
                        img_name = line.split('\t')[0]
                        rows.append({'img_name': img_name, 'label': label})

        elif type == 'val':
            '''
            进入 val 下的 txt, 把前两列读出来, 然后用 mapping 换为 label 即可
            '''
            with open(os.path.join(self.root, 'val', 'val_annotations.txt'), 'r') as f:
                lines = f.readlines()
                for line in lines:
                    img_name, id, _, _, _ = line.split('\t')
                    label = mapping[id]
                    rows.append({'img_name': img_name, 'label': label})

        df = pd.concat([df, pd.DataFrame(rows)], ignore_index=True)
        print(len(df))
        import sys
        sys.exit()
        return df
    
    def __len__(self):
        return len(self.annotations_file)
    
    def __getitem__(self, index):
        pass
        
train_data = TinyImagenet(root="./data/tiny-imagenet-200", type="train")
        

TypeError: get_annotations_file() takes 1 positional argument but 2 were given

In [ ]:
# 任务:
# 1. 读入 FashionMNIST, CIFAR10, Tiny Imagenet(来自CS231n课程项目) 数据集
# 2. 构建 CNN 网络, 写 train/eval loop, 带调试信息
# 3. 训练, 保存, 加载

# 数据集导入
from torchvision import datasets, transforms

op_dataset = 1  # 0: FashionMNIST, 1: CIFAR10, 2: TinyImagenet
op_model = 0    # 0: LetNet

if op_dataset == 0:
    train_data = datasets.FashionMNIST(
        root='./data',
        train=True,
        download=True,
        transform=transforms.ToTensor(),
    )
    test_data = datasets.FashionMNIST(
        root='./data',
        train=False,
        download=True,
        transform=transforms.ToTensor(),
    )
elif op_dataset == 1:
    train_data = datasets.CIFAR10(
        root='./data',
        train=True,
        download=True,
        transform=transforms.ToTensor(),
    )
    test_data = datasets.CIFAR10(
        root='./data',
        train=False,
        download=True,
        transform=transforms.ToTensor(),
    )
elif op_dataset == 2:
    train_data = TinyImagenet('./data/tiny-imagenet-200', 'train')
    test_data = TinyImagenet('./data/tiny-imagenet-200', 'val')

from torch.utils.data import DataLoader

batch_size = 64
train_dataloader = DataLoader(train_data, batch_size= batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
print('训练集和测试集的样本数量')
print(len(train_data), len(test_data))

Files already downloaded and verified
Files already downloaded and verified
训练集和测试集的样本数量
50000 10000


In [7]:
# LetNet
from torch import nn
import torch

sample = train_data[0][0]

class LetNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=sample.shape[0], out_channels=6, kernel_size=5)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(self.get_flatten_dim(), 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)    # BUG: 这里如果是Tiny Imagenet, 10改为200

    def get_flatten_dim(self):
        with torch.no_grad():
            x = self.pool(self.conv1(sample.unsqueeze(0)))
            x = self.pool(self.conv2(x))
            return x.numel()
        
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        logits = self.fc3(x)
        return logits

In [8]:
# train/eval

from torch import optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'
if op_model == 0:
    model = LetNet().to(device)
    model_name = 'LetNet'
    
loss_fn = nn.CrossEntropyLoss()
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)

def train_loop(model, dataloader, loss_fn, optimizer):
    model.train()
    samples_size = len(dataloader.dataset)
    batches_size = len(dataloader)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        logits_pred = model(X)
        loss = loss_fn(logits_pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch % 100 == 0 or batch == batches_size - 1:
            print(f'loss:{loss:>7.2f} | {batch * batch_size + len(y)}/{samples_size}')

def test_loop(model, dataloader, loss_fn):
    model.eval()
    samples_size = len(dataloader.dataset)
    batches_size = len(dataloader)
    loss_sum = 0
    acc_sum = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        with torch.no_grad():
            logits_pred = model(X)
        loss_sum += loss_fn(logits_pred, y)
        acc_sum += (logits_pred.argmax(dim=1) == y).sum()
    print(f'loss:{(loss_sum/batches_size):>7.2f} | acc:{(acc_sum/samples_size*100):>7.2f}%')

epochs = 10
for t in range(1, epochs + 1):
    print(f'Epoch {t} -------------------------')
    train_loop(model, train_dataloader, loss_fn, optimizer)
    test_loop(model, test_dataloader, loss_fn)
    print()

# 保存/加载模型
import os

if not os.path.exists('./model'):
    os.makedirs('./model')
torch.save(model.state_dict(), os.path.join('./model', model_name + '.path'))

model_new = LetNet().to(device)
model_new.load_state_dict(torch.load(os.path.join('./model', model_name + '.path')))
model_new.eval()
with torch.no_grad():
    y_pred = model_new(sample.unsqueeze(0).to(device)).argmax()
print(y_pred, train_data[0][1])

Epoch 1 -------------------------
loss:   2.31 | 64/50000
loss:   2.08 | 6464/50000
loss:   1.91 | 12864/50000
loss:   1.80 | 19264/50000
loss:   1.51 | 25664/50000
loss:   1.59 | 32064/50000
loss:   1.56 | 38464/50000
loss:   1.68 | 44864/50000
loss:   1.61 | 50000/50000
loss:   1.49 | acc:  45.67%

Epoch 2 -------------------------
loss:   1.42 | 64/50000
loss:   1.45 | 6464/50000
loss:   1.54 | 12864/50000
loss:   1.19 | 19264/50000
loss:   1.42 | 25664/50000
loss:   1.24 | 32064/50000
loss:   1.32 | 38464/50000
loss:   1.48 | 44864/50000
loss:   1.36 | 50000/50000
loss:   1.39 | acc:  49.67%

Epoch 3 -------------------------
loss:   1.22 | 64/50000
loss:   1.38 | 6464/50000
loss:   1.38 | 12864/50000
loss:   1.39 | 19264/50000
loss:   1.19 | 25664/50000
loss:   1.76 | 32064/50000
loss:   1.19 | 38464/50000
loss:   1.41 | 44864/50000
loss:   0.90 | 50000/50000
loss:   1.31 | acc:  52.01%

Epoch 4 -------------------------
loss:   0.99 | 64/50000
loss:   1.45 | 6464/50000
loss:   1.

NameError: name 'model_name' is not defined